## Load the libraries we need

In [ ]:
import pandas as pd
import numpy as np
import os
from bs4 import BeautifulSoup

## User settings

In [ ]:
year = 2022

filename_cbp = os.environ['BDS_HOME'] + '/badassdatascience/econometrics/church_to_bar_ratio/' + str(year) + '/data' + '/CBP2022.CB2200CBP-Data.csv'
output_directory = os.environ['BDS_HOME'] + '/badassdatascience/econometrics/church_to_bar_ratio/' + str(year) + '/output'
data_directory = os.environ['BDS_HOME'] + '/badassdatascience/econometrics/church_to_bar_ratio/' + str(year) + '/data'

output_file = output_directory + '/church_to_bar_ratio_by_US_county_' + str(year) + '.svg'

# these are imported as strings due to non-numeric characters in a few rows for this column
naics_drinking_places = '722410'
naics_religious_organizations = '813110'

number_of_groups = 9
map_svg = data_directory + '/Usa_counties_large.svg'
colors = ['#000000', '#F7F4F9', '#E7E1EF', '#D4B9DA', '#C994C7', '#DF65B0', '#E7298A', '#CE1256', '#980043', '#67001F']

path_style = 'font-size:12px;fill-rule:nonzero;stroke:#FFFFFF;stroke-opacity:1;stroke-width:0.1;stroke-miterlimit:4;stroke-dasharray:none;stroke-linecap:butt;marker-start:none;stroke-linejoin:bevel;fill:'

## Define a function to obtain the FIPS code from a GEO_ID

In [ ]:
def get_FIPS(item):
    new_item = item.strip().split('US')[-1]
    if len(new_item) < 5:
        return np.nan
    else:
        return new_item
    return new_item

## Load and pre-process the County Business Patterns data

As of 2025-01-18, the most recent year the US Census Bureau put this information out is 2022.

In [ ]:
naics_list = [naics_drinking_places, naics_religious_organizations]

df = (
    pd.read_csv(filename_cbp, low_memory=False)
    .iloc[1:, :]
    .astype(
        {
            'ESTAB' : 'int64',
            'YEAR' : 'int64',
        }
    )
)

df = df[(df['YEAR'] == year) & (df['NAICS2017'].isin(naics_list))]
df['FIPS'] = df['GEO_ID'].apply(get_FIPS)
df = df[['FIPS', 'NAICS2017', 'ESTAB']]
df = df.dropna()

In [ ]:
df

## Aggregate to compute the count of establishments per (FIPS, NAICS) pair

It looks like the NAICS values considered were specified in 2017 and the source data explicitly says so in that columns header name:

In [ ]:
df_agg = (
    df
    .groupby(['FIPS', 'NAICS2017'])
    ['ESTAB']
    .agg(
        'sum'
    )
)

df_agg = df_agg.reset_index()

In [ ]:
df_agg

## Divide the set by NAICS to prepare for join by FIPS code

I suppose we could create a pivot table instead for this task, but this works and is easy to understand:

In [ ]:
df_drinking_places = df_agg[df_agg['NAICS2017'] == naics_drinking_places].copy()
df_drinking_places['drinking_places'] = df_drinking_places['ESTAB']
df_drinking_places = df_drinking_places.drop(columns = ['ESTAB']).sort_values(by = ['FIPS'])

df_religious_organizations = df_agg[df_agg['NAICS2017'] == naics_religious_organizations].copy()
df_religious_organizations['religious_organizations'] = df_religious_organizations['ESTAB']
df_religious_organizations = df_religious_organizations.drop(columns = ['ESTAB']).sort_values(by = ['FIPS'])

In [ ]:
df_drinking_places

In [ ]:
df_religious_organizations

## Join so we obtain both counts per FIPS code on the same row

In [ ]:
df_cbr = pd.merge(
    df_religious_organizations[['FIPS', 'religious_organizations']],
    df_drinking_places[['FIPS', 'drinking_places']],
    on = ['FIPS'],
    how = 'left',
).sort_values(by = ['FIPS'])

In [ ]:
df_cbr

## Add a pseudocount to prevent divide by zero issues

We first fill the NaN values with zeros.

In [ ]:
df_cbr = df_cbr.fillna(0.)
df_cbr['religious_organizations'] = df_cbr['religious_organizations'] + 1.
df_cbr['drinking_places'] = df_cbr['drinking_places'] + 1.

In [ ]:
df_cbr

## Compute the church to bar ratio and its log10

In [ ]:
df_cbr['church_to_bar_ratio'] = df_cbr['religious_organizations'] / df_cbr['drinking_places']
df_cbr['log10_church_to_bar_ratio'] = np.log10(df_cbr['church_to_bar_ratio'])

In [ ]:
df_cbr

## QA

#### Should be empty

In [ ]:
df_cbr[df_cbr['church_to_bar_ratio'].isna()]

In [ ]:
df_cbr[df_cbr['log10_church_to_bar_ratio'].isna()]

#### Review the value ranges

In [ ]:
print(np.min(df_cbr['church_to_bar_ratio']), np.max(df_cbr['church_to_bar_ratio']))

In [ ]:
print(np.min(df_cbr['log10_church_to_bar_ratio']), np.max(df_cbr['log10_church_to_bar_ratio']))

## Shift the values so no negative ones remain

In [ ]:
min_log10 = np.min(df_cbr['log10_church_to_bar_ratio'])
df_cbr['zeroed_log10_church_to_bar_ratio'] = df_cbr['log10_church_to_bar_ratio'] - min_log10

## Define the color cutpoints

In [ ]:
color_interval = np.max(df_cbr['zeroed_log10_church_to_bar_ratio']) / np.float64(number_of_groups)

cutpoints = []
for n in range(0, np.int32(number_of_groups)):
    start = color_interval * np.float64(n)
    end = color_interval * np.float64(n + 1)
    cutpoints.append( (n, start, end) )

In [ ]:
cutpoints

## Assign the per-county color using the cutpoints

In [ ]:
def assign_color_group(item):
    for n, start, end in cutpoints:
        if item >= start and item <= end:
            return np.int32(n)

df_cbr['color_group'] = df_cbr['zeroed_log10_church_to_bar_ratio'].apply(assign_color_group)

## More QA

#### DataFrame should now have a color assignment for each row

In [ ]:
df_cbr

#### Should report an empty set

I.e., there are no NaNs in the "color_group" column:

In [ ]:
df_cbr[df_cbr['color_group'].isna()]

## Create a lookup dictionary matching FIPS to color

There might be a more elegant way to do this completely in Pandas, but that is left as an exercise for the reader!

In [ ]:
color_group = {}
for v, g in zip(df_cbr['FIPS'], df_cbr['color_group']):
    color_group[v] = g

## Create the map

In [ ]:
svg = open(map_svg, 'r').read()
soup = BeautifulSoup(svg, selfClosingTags=['defs','sodipodi:namedview'])
paths = soup.findAll('path')
colors.reverse()

for p in paths:

    plot = False
    if p['class'][0].replace('c', '') in color_group:
        color_class = color_group[p['class'][0].replace('c', '')]
        plot = True

    elif p['class'][0] == 'c':
        color_class = 9
        plot = True

    if plot:
        color = colors[color_class]
        p['style'] = path_style + color

## Save the map

In [ ]:
with open(output_file, 'w') as f:
    f.write(str(soup.prettify()))

## Display the map

In [ ]:
from IPython.display import SVG
SVG(output_file)